In [30]:
import pandas as pd
import numpy as np
import nltk
import re
import geonamescache
import seaborn as sns

In [31]:
df = pd.read_csv('ndc.csv')
df = df.dropna()
print(df)

                              name
0                       Row Labels
1                SPIRONOLACTONE % 
2        -L'Oreal USA Products Inc
3                 .Cardinal Health
4     .Church & Dwight Canada Corp
...                            ...
7022                        ZYGONE
7023    Zyla Life Sciences US Inc.
7024                        #NAME?
7025                       (blank)
7026                   Grand Total

[7027 rows x 1 columns]


# Make everything lower case

In [32]:
df['lower_name'] = df['name'].str.lower()
df

,name,lower_name
0,Row Labels,row labels
1,SPIRONOLACTONE %,spironolactone %
2,-L'Oreal USA Products Inc,-l'oreal usa products inc
3,.Cardinal Health,.cardinal health
4,.Church & Dwight Canada Corp,.church & dwight canada corp
...,...,...
7022,ZYGONE,zygone
7023,Zyla Life Sciences US Inc.,zyla life sciences us inc.
7024,#NAME?,#name?
7025,(blank),(blank)


# Remove all non-alphanumeric characters except white spaces

In [33]:
df['filter_alpha_num'] = df['lower_name'].apply(lambda x: re.sub(r'[^a-zA-Z\d\s:]', '', x))
df

,name,lower_name,filter_alpha_num
0,Row Labels,row labels,row labels
1,SPIRONOLACTONE %,spironolactone %,spironolactone
2,-L'Oreal USA Products Inc,-l'oreal usa products inc,loreal usa products inc
3,.Cardinal Health,.cardinal health,cardinal health
4,.Church & Dwight Canada Corp,.church & dwight canada corp,church dwight canada corp
...,...,...,...
7022,ZYGONE,zygone,zygone
7023,Zyla Life Sciences US Inc.,zyla life sciences us inc.,zyla life sciences us inc
7024,#NAME?,#name?,name
7025,(blank),(blank),blank


# Remove common company abbreviations

In [34]:
company_abbrev = {
    "labs",
    "bio",
    "biotech",
    "laboratories",
    "ventures",
    "solutions",
    "sciences",
    "partners",
    "holdings",
    "son",
    "co",
    "corp",
    "corporation",
    "inc",
    "incorporated",
    "brothers",
    "tech",
    "foods",
    "books",
    "industrial",
    "innovations",
    "bureau",
    "prestige",
    "worldwide",
    "international",
    "direct",
    "online",
    "properties",
    "pharm",
    "pharms",
    "pharma",
    "usa",
    "development",
    "technologies",
    "consulting",
    "works",
    "unlimited",
    "specialties",
    "scintific",
    "digital",
    "brands",
    "logistics",
    "llc",
    "ltd",
    "companies",
    "company",
    "creative",
    "productions",
    "prod",
}

def filter_company_abbrev(company_name):
    return " ".join([v for v in nltk.word_tokenize(company_name) if not v in company_abbrev])

df['filter_comp_abbrev'] = df['filter_alpha_num'].map(filter_company_abbrev)
df

,name,lower_name,filter_alpha_num,filter_comp_abbrev
0,Row Labels,row labels,row labels,row labels
1,SPIRONOLACTONE %,spironolactone %,spironolactone,spironolactone
2,-L'Oreal USA Products Inc,-l'oreal usa products inc,loreal usa products inc,loreal products
3,.Cardinal Health,.cardinal health,cardinal health,cardinal health
4,.Church & Dwight Canada Corp,.church & dwight canada corp,church dwight canada corp,church dwight canada
...,...,...,...,...
7022,ZYGONE,zygone,zygone,zygone
7023,Zyla Life Sciences US Inc.,zyla life sciences us inc.,zyla life sciences us inc,zyla life us
7024,#NAME?,#name?,name,name
7025,(blank),(blank),blank,blank


# Remove countries and cities information

In [35]:
gc = geonamescache.GeonamesCache()

continents = gc.get_continents()
# print(continents)
continents = [continents[key]['name'].lower() for key in continents]

countries = gc.get_countries()
countries = [countries[key]['name'].lower() for key in countries]

cities = gc.get_cities()
cities = [cities[key]['name'].lower() for key in cities]

continents.extend(countries)
continents.extend(cities)
locations = continents

# print('usa' in locations) # False
# print('us' in locations) # False
# print('united states' in locations) # True
# Adding some abbreviations
locations.extend(['usa', 'us'])

locations = set(locations)

In [36]:
def filter_locations(company_name):
    return " ".join([v for v in nltk.word_tokenize(company_name) if not v in locations])

print(filter_locations('loreal usa products inc'))

df['filter_loc'] = df['filter_comp_abbrev'].apply(lambda x: filter_locations(x))
df

loreal products inc


,name,lower_name,filter_alpha_num,filter_comp_abbrev,filter_loc
0,Row Labels,row labels,row labels,row labels,row labels
1,SPIRONOLACTONE %,spironolactone %,spironolactone,spironolactone,spironolactone
2,-L'Oreal USA Products Inc,-l'oreal usa products inc,loreal usa products inc,loreal products,loreal products
3,.Cardinal Health,.cardinal health,cardinal health,cardinal health,cardinal health
4,.Church & Dwight Canada Corp,.church & dwight canada corp,church dwight canada corp,church dwight canada,church dwight
...,...,...,...,...,...
7022,ZYGONE,zygone,zygone,zygone,zygone
7023,Zyla Life Sciences US Inc.,zyla life sciences us inc.,zyla life sciences us inc,zyla life us,zyla life
7024,#NAME?,#name?,name,name,name
7025,(blank),(blank),blank,blank,blank


# Remove all common dictionary terms (and their plural forms)

In [37]:
from nltk.corpus import words

dictionary_words = words.words()
dictionary_words = set([v for v in dictionary_words if len(v) > 1])

p = nltk.PorterStemmer()

In [38]:
# def filter_common_words(company_name):
#     # print([v for v in nltk.word_tokenize(company_name) if v in dictionary_words])
#     return " ".join([v for v in nltk.word_tokenize(company_name) if not p.stem(v) in dictionary_words])

# df['filter_common'] = df['filter_loc'].apply(lambda x: filter_common_words(x))
# df

# Select the shortest string among the columns

In [39]:
def get_shortest(*args):
    return min([v for v in args if len(v) > 0], key=len)
    
df['cleaned'] = df.apply(lambda x: get_shortest(x['lower_name'], x['filter_alpha_num'], x['filter_loc'], x['filter_comp_abbrev']), axis=1)

In [40]:
df.sample(50)

,name,lower_name,filter_alpha_num,filter_comp_abbrev,filter_loc,cleaned
143,Advanced Healthcare Solutions,advanced healthcare solutions,advanced healthcare solutions,advanced healthcare,advanced healthcare,advanced healthcare
2109,"e Pharma, LLC","e pharma, llc",e pharma llc,e,e,e
2190,"EltaMD, Inc","eltamd, inc",eltamd inc,eltamd,eltamd,eltamd
6408,Tri Lakes Home Medical Eq. Inc.,tri lakes home medical eq. inc.,tri lakes home medical eq inc,tri lakes home medical eq,tri lakes home medical eq,tri lakes home medical eq
6380,"Topical BioMedics, Inc.","topical biomedics, inc.",topical biomedics inc,topical biomedics,topical biomedics,topical biomedics
6280,The University of Utah DBA Cyclotron Radiochem...,the university of utah dba cyclotron radiochem...,the university of utah dba cyclotron radiochem...,the university of utah dba cyclotron radiochem...,the utah dba cyclotron radiochemistry lab,the utah dba cyclotron radiochemistry lab
4255,Mylan Pharmaceuticals Inc.,mylan pharmaceuticals inc.,mylan pharmaceuticals inc,mylan pharmaceuticals,mylan pharmaceuticals,mylan pharmaceuticals
3106,"Hyatt Life Sciences, Inc.","hyatt life sciences, inc.",hyatt life sciences inc,hyatt life,hyatt life,hyatt life
472,ANDA,anda,anda,anda,,anda
1458,Chain Drug Marketing,chain drug marketing,chain drug marketing,chain drug marketing,chain drug marketing,chain drug marketing
